In [0]:
%run ./Log_Files

In [0]:
log_event_file(
    pipeline_name="Patient_Insurance_SCD2",
    notebook_name="Base_table",
    layer_name="Gold",
    status="STARTED",
    message="Gold SCD2 pipeline started",
    record_count=0,
    event_time=datetime.now()
)

**UPDATION OF PATIENT'S RECORDS**

In [0]:
# from datetime import datetime

# def log_event(layer, table_name, event, record_count=0):
#     log_df = spark.createDataFrame(
#         [(layer,
#           table_name,
#           event,
#           datetime.now().isoformat(),
#           record_count
#           )
#         ],
#         ["layer", "table_name", "event", "event_time", "record_count"]
#     )

#     log_df.write.mode("append").saveAsTable("logs")

In [0]:
# log_event(
#     layer="GOLD",
#     table_name="gold_patient_dtable",
#     event="START"
# )

In [0]:

try:
    spark.sql("""
    MERGE INTO gold_patient_scd2_dtable t USING silver_patient_dtable s
    ON t.PATIENT_KEY = s.PATIENT_KEY
    AND t.CURR_RCD_IND = 'Y'

    WHEN MATCHED
    AND (
            s.ACTIONIND = 'D'
         OR t.CHK_SUM_TXT <> s.CHK_SUM_TXT
        )
    THEN UPDATE SET
        t.CURR_RCD_IND = 'N',
        t.UPDT_TS = current_timestamp()
    """)
    spark.sql("""INSERT INTO gold_patient_scd2_dtable
    SELECT
        s.PATIENTID,
        s.FIRST,
        s.LAST,
        s.FULL_NAME,
        s.BIRTHDATE,
        s.DEATHDATE,
        s.MARITAL,
        s.GENDER,
        s.ADDRESS,
        s.CITY,
        s.STATE,
        s.COUNTRY,
        s.HOSPITALNAME,
        s.REASONDESCRIPTION,
        s.PAYERID,
        CASE 
            WHEN s.ACTIONIND = 'D' THEN 'N'
            ELSE 'Y'
        END AS CURR_RCD_IND,
        current_timestamp() AS CRET_TS,
        CASE 
            WHEN s.ACTIONIND = 'D' THEN current_timestamp()
            ELSE TIMESTAMP('9999-12-31')
        END AS UPDT_TS,
        s.LOAD_CTL_KEY,
        s.PATIENT_KEY,
        s.CHK_SUM_TXT
    FROM silver_patient_dtable s
    LEFT JOIN gold_patient_scd2_dtable g
        ON s.PATIENT_KEY = g.PATIENT_KEY
    WHERE 
          g.PATIENT_KEY IS NULL
       OR (
            g.CHK_SUM_TXT <> s.CHK_SUM_TXT
            AND s.ACTIONIND <> 'D'
          )
    """)
    record_count = spark.table("gold_patient_scd2_dtable").count()

    log_event_file(
        pipeline_name="Patient_Insurance_SCD2",
        notebook_name="Base_table",
        layer_name="Gold",
        status="SUCCESS",
        message="Gold SCD2 pipeline completed successfully",
        record_count=record_count,
        event_time=datetime.now()
    )
except Exception as e:

    log_event_file(
        pipeline_name="Patient_Insurance_SCD2",
        notebook_name="Base_table",
        layer_name="Gold",
        status="FAILED",
        message=str(e),
        record_count=0,
        event_time=datetime.now()
    )

    raise e





In [0]:
%sql
select * from gold_patient_scd2_dtable where CURR_RCD_IND = 'N'

In [0]:
%sql
select * from gold_patient_scd2_dtable

In [0]:
records_read = spark.table("gold_patient_scd2_dtable").count()

log_event_file(
     pipeline_name="Patient_Insurance_SCD2",
     notebook_name="Base_table",
     layer_name="Gold",
     status="END",
     message=f"Gold pipeline Finished",
     record_count=records_read,
event_time=datetime.now()
 )


In [0]:
# %sql
# -- STEP 2: INSERT NEW + UPDATED ACTIVE RECORDS
# INSERT INTO gold_patient_scd2_dtable
# SELECT
#     s.PATIENTID,
#     s.FIRST,
#     s.LAST,
#     s.FULL_NAME,
#     s.BIRTHDATE,
#     s.DEATHDATE,
#     s.MARITAL,
#     s.GENDER,
#     s.ADDRESS,
#     s.CITY,
#     s.STATE,
#     s.COUNTRY,
#     s.HOSPITALNAME,
#     s.REASONDESCRIPTION,
#     s.PAYERID,
#     CASE 
#         WHEN s.ACTIONIND = 'D' THEN 'N'
#         ELSE 'Y'
#     END AS CURR_RCD_IND,
#     current_timestamp() AS CRET_TS,
#     CASE 
#         WHEN s.ACTIONIND = 'D' THEN current_timestamp()
#         ELSE TIMESTAMP('9999-12-31')
#     END AS UPDT_TS,
#     s.LOAD_CTL_KEY,
#     s.PATIENT_KEY,
#     s.CHK_SUM_TXT
# FROM silver_patient_dtable s
# LEFT JOIN gold_patient_scd2_dtable g
#     ON s.PATIENT_KEY = g.PATIENT_KEY
# WHERE 
#       g.PATIENT_KEY IS NULL
#    OR (
#         g.CHK_SUM_TXT <> s.CHK_SUM_TXT
#         AND s.ACTIONIND <> 'D'
#       );


In [0]:
%sql
select * from gold_patient_scd2_dtable

In [0]:
# gold_count = spark.table("gold_patient_scd2_dtable").count()

# log_event(
#     layer="GOLD",
#     table_name="gold_patient_scd2_dtable",
#     event="END",
#     record_count=gold_count
# )


In [0]:
# %sql
# select * from gold_patient_scd2_dtable version as of 5

In [0]:
%sql
SELECT * FROM gold_patient_scd2_dtable VERSION AS OF 5;

In [0]:
%sql
DESCRIBE HISTORY gold_patient_scd2_dtable

In [0]:
%sql
select * from gold_patient_scd2_dtable

**UPDATION OF PAYER'S RECORDS**

In [0]:
%sql
-- STEP 1: CLOSE OLD RECORD (UPDATE)
MERGE INTO gold_payer_scd2_dtable AS t
USING silver_payer_dtable AS s
ON t.PAYER_KEY = s.PAYER_KEY
AND t.CURR_RCD_IND = 1

WHEN MATCHED AND t.CHK_SUM_TXT <> s.CHK_SUM_TXT THEN
UPDATE SET
    t.CURR_RCD_IND = 0,
    t.TransactionCode = 'U',
    t.ActivationInd="Inactive",
    t.UPDT_TS = current_timestamp()
;


In [0]:
%sql
select * from gold_payer_scd2_dtable where TransactionCode = 'U'

In [0]:
%sql
-- STEP 2: INSERT NEW + UPDATED ACTIVE RECORDS
INSERT INTO gold_payer_scd2_dtable
SELECT
    s.PAYERID,
    s.NAME,
    s.ADDRESS,
    s.CITY,
    s.STATE_HEADQUARTERED,
    s.ZIP,
    s.PHONE,
    s.PAYER_KEY,
    s.CHK_SUM_TXT,
    CASE 
        WHEN g.CHK_SUM_TXT IS NULL THEN 'I'   -- brand new
        WHEN g.CHK_SUM_TXT <> s.CHK_SUM_TXT THEN 'I'  -- new version
    END AS TransactionCode,
    1 AS CURR_RCD_IND,
    s.ActivationInd,
    current_timestamp() AS CRET_TS,
    TIMESTAMP('9999-12-31') AS UPDT_TS,
    s.LOAD_CTL_KEY
FROM silver_payer_dtable s
LEFT JOIN gold_payer_scd2_dtable g
    ON s.PAYER_KEY = g.PAYER_KEY 
    --AND g.CURR_RCD_IND = 1
WHERE 
      g.CHK_SUM_TXT IS NULL            -- new
   OR g.CHK_SUM_TXT <> s.CHK_SUM_TXT;  -- changed


### **DELETION OF INACTIVE PAYER RECORDS

SOFT DELETE**

In [0]:
%sql
-- STEP 0: SOFT DELETE ONLY INACTIVE RECORDS FROM SOURCE
MERGE INTO gold_payer_scd2_dtable AS t
USING silver_payer_dtable AS s
ON t.PAYER_KEY = s.PAYER_KEY
   --AND t.CURR_RCD_IND = 1          -- Only active records in GOLD

WHEN MATCHED 
     AND s.ActivationInd = 'Inactive'   -- Only if source marks it INACTIVE
THEN 
  UPDATE SET
        --t.CURR_RCD_IND = 0,
        --t.ActivationInd = 'Inactive',
        t.TransactionCode = 'D',   -- D = Deleted / Inactive
        t.UPDT_TS = current_timestamp();


In [0]:
%sql
select * from gold_payer_scd2_dtable where TransactionCode = 'D'

**HARD DELETE**

In [0]:
%sql
--DELETE FROM gold_payer_scd2_dtable
--WHERE TransactionCode = 'D';

In [0]:
%sql
select * from gold_payer_scd2_dtable